In [10]:
import numpy as np
import sys
import cv2
from math import pow, sqrt

In [11]:
labels = [line.strip() for line in open("class_labels.txt")]

In [12]:
# Load model
print("\nLoading model...\n")

network = cv2.dnn.readNetFromCaffe("SSD_MobileNet_prototxt.txt","SSD_MobileNet.caffemodel")

print("\nStreaming video using device...\n")



# Capture video from file or through device
cap = cv2.VideoCapture("vtest.avi")

frame_no = 0

while cap.isOpened():

    frame_no = frame_no+1

    # Capture one frame after another
    ret, frame = cap.read()

    if not ret:
        break

    (h, w) = frame.shape[:2]

    # Resize the frame to suite the model requirements. Resize the frame to 300X300 pixels
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 0.007843, (300, 300), 127.5)

    network.setInput(blob)
    detections = network.forward()

    pos_dict = dict()
    coordinates = dict()



    for i in range(detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > 0.2:

            class_id = int(detections[0, 0, i, 1])

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype('int')

            # Filtering only persons detected in the frame. Class Id of 'person' is 15
            if class_id == 15.00:


                coordinates[i] = (startX, startY, endX, endY)

                # Mid point of bounding box
                x_mid = round((startX+endX)/2,4)
                y_mid = round((startY+endY)/2,4)

                height = round(endY-startY,4)

                # Distance from camera based on triangle similarity
                distance = (165 * F)/height
                print("Distance(cm):{dist}\n".format(dist=distance))

                # Mid-point of bounding boxes (in cm) based on triangle similarity technique
                x_mid_cm = (x_mid * distance) / F
                y_mid_cm = (y_mid * distance) / F
                pos_dict[i] = (x_mid_cm,y_mid_cm,distance)

    # Distance between every object detected in a frame
    close_objects = set()
    for i in pos_dict.keys():
        for j in pos_dict.keys():
            if i < j:
                dist = sqrt(pow(pos_dict[i][0]-pos_dict[j][0],2) + pow(pos_dict[i][1]-pos_dict[j][1],2) + pow(pos_dict[i][2]-pos_dict[j][2],2))

                # Check if distance less than 2 metres or 200 centimetres
                if dist < 200:
                    close_objects.add(i)
                    close_objects.add(j)

    for i in pos_dict.keys():
        if i in close_objects:
            COLOR = (0,0,255)
        else:
            COLOR = (0,255,0)
        (startX, startY, endX, endY) = coordinates[i]

        cv2.rectangle(frame, (startX, startY), (endX, endY), COLOR, 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        # Convert cms to feet
        cv2.putText(frame, 'Depth: {i} ft'.format(i=round(pos_dict[i][2]/30.48,4)), (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLOR, 2)

    cv2.namedWindow('Frame',cv2.WINDOW_NORMAL)

    # Show frame
    cv2.imshow('Frame', frame)
    cv2.resizeWindow('Frame',800,600)

    key = cv2.waitKey(1) & 0xFF

    # Press `q` to exit
    if key == ord("q"):
        break

# Clean
cap.release()
cv2.destroyAllWindows()



Loading model...


Streaming video using device...

Distance(cm):994.8529411764706

Distance(cm):1300.9615384615386

Distance(cm):994.8529411764706

Distance(cm):1252.7777777777778

Distance(cm):1014.75

Distance(cm):1179.9418604651162

Distance(cm):1025.0

Distance(cm):1179.9418604651162

Distance(cm):1193.8235294117646

Distance(cm):1166.3793103448277

Distance(cm):1127.5

Distance(cm):1317.857142857143

Distance(cm):1153.125

Distance(cm):1140.1685393258426

Distance(cm):1193.8235294117646

Distance(cm):1035.4591836734694

Distance(cm):1140.1685393258426

Distance(cm):1208.0357142857142

Distance(cm):867.3076923076923

Distance(cm):1068.157894736842

Distance(cm):914.1891891891892

Distance(cm):1166.3793103448277

Distance(cm):1102.9891304347825

Distance(cm):1079.5212765957447

Distance(cm):852.7310924369748

